In [1]:
import pandas as pd

In [56]:
df_high_price = pd.DataFrame({
    'Sales': [50000, 30000, 20000],
    'High': [0.2, 0.25, 0.55],
    'Medium': [0.25, 0.3, 0.45],
    'Low': [0.3, 0.35, 0.35]
})
df_medium_price = pd.DataFrame({
    'Sales': [50000, 30000, 20000],
    'High': [0.25, 0.35, 0.40],
    'Medium': [0.30, 0.40, 0.30],
    'Low': [0.40, 0.50, 0.10]
})
df_low_price = pd.DataFrame({
    'Sales': [50000, 30000, 20000],
    'High': [0.35, 0.40, 0.25],
    'Medium': [0.40, 0.50, 0.10],
    'Low': [0.50, 0.45, 0.05]
})
sales_dfs = {
    "Low": df_low_price,
    "High" : df_high_price,
    "Medium" : df_medium_price
}
price = {
    "Low" : 30,
    "Medium" : 40,
    "High" : 50
}
compe_prob = {
    "Low" : 0.1,
    "Medium" : 0.7,
    "High" : 0.2
}

## First Fold

Competion Focused

In [57]:
fold_first_compe = {}
for keys,df_sale in sales_dfs.items():
    temp = {}
    for fold_col in price.keys():
        df_sale[f'1st_fold_{fold_col}Comp'] = df_sale.loc[:,fold_col] * df_sale.loc[:,'Sales']
        temp[f'{fold_col}_compe'] = df_sale[f'1st_fold_{fold_col}Comp'].sum()
    fold_first_compe[f'{keys}_price'] = temp

fold_first_compe

{'Low_price': {'Low_compe': 39500.0,
  'Medium_compe': 37000.0,
  'High_compe': 34500.0},
 'High_price': {'Low_compe': 32500.0,
  'Medium_compe': 30500.0,
  'High_compe': 28500.0},
 'Medium_price': {'Low_compe': 37000.0,
  'Medium_compe': 33000.0,
  'High_compe': 31000.0}}

# Second Fold
Unit Sold focused

In [ ]:
sec_fold = {}
for key_price, item_price in fold_first_compe.items():
    sum = 0
    for key_compe, item_compe in  item_price.items():
        stats_compe = key_compe.split("_")[0]
        sum += compe_prob[stats_compe]*item_compe
    sec_fold[key_price] = sum
sec_fold

{'Low_price': 36750.0, 'High_price': 30300.0, 'Medium_price': 33000.0}

## Third Fold
Profit

In [69]:
thir_fold = {}
for key_price, unit_sold in sec_fold.items():
    stats_price = key_price.split("_")[0]
    print(f"{unit_sold} unit X ${price[stats_price]} = ${unit_sold* price[stats_price]}")
    thir_fold[key_price] = unit_sold* price[stats_price]
thir_fold

36750.0 unit X $30 = $1102500.0
30300.0 unit X $50 = $1515000.0
33000.0 unit X $40 = $1320000.0


{'Low_price': 1102500.0, 'High_price': 1515000.0, 'Medium_price': 1320000.0}

In [72]:
import numpy as np
max_key = 0
max_val = 0

for key_price, item_profit in thir_fold.items():
    if item_profit > max_val:
        max_val = item_profit
        max_key = key_price

print(f"Optimal Decision: {max_key} with {max_val}" )

Optimal Decision: High_price with 1515000.0


---
## Research or No
For no, retain score above.
For yes,...


In [61]:
df_research_prob = pd.DataFrame({
    "hC" : [0.80, 0.15, 0.05],
    "mC" : [0.15,0.80, 0.05],
    "lC" : [0.03,0.07, 0.90]
}, index=["P-hC", "P-mC", "P-lC"])
df_research_prob

,hC,mC,lC
P-hC,0.80,0.15,0.03
P-mC,0.15,0.80,0.07
P-lC,0.05,0.05,0.90


In [62]:
compe_prob

{'Low': 0.1, 'Medium': 0.7, 'High': 0.2}

In [63]:
compe_prob_vals = np.array(list(compe_prob.values()))[::-1]
compe_prob_vals

array([0.2, 0.7, 0.1])

In [64]:
Preds_prob = df_research_prob.apply(lambda X: X.values * compe_prob_vals, axis=1)
Preds_prob = pd.DataFrame(Preds_prob.to_dict(), index=['hc', 'mc', 'lc']).T
Preds_prob

,hc,mc,lc
P-hC,0.16,0.105,0.003
P-mC,0.03,0.560,0.007
P-lC,0.01,0.035,0.090


In [53]:
preds_total_preds = Preds_prob.sum(axis=1)
preds_total_preds

P-hC    0.268
P-mC    0.597
P-lC    0.135
dtype: float64

In [54]:
bayesian = Preds_prob.apply(lambda X: X.values /preds_total_preds[X.name], axis=1).to_dict()
bayesian_df = pd.DataFrame(bayesian, index=['hc', 'mc', 'lc'])
bayesian_df

,P-hC,P-mC,P-lC
hc,0.597015,0.050251,0.074074
mc,0.391791,0.938023,0.259259
lc,0.011194,0.011725,0.666667


## Research First Fold

array([28500., 30500., 32500.])

In [84]:
sales_count = pd.Series(np.array(list(fold_first_compe['High_price'].values()))[::-1], index=['hc', 'mc', 'lc'])
sales_count

hc    28500.0
mc    30500.0
lc    32500.0
dtype: float64

In [91]:
bayesian_trans = bayesian_df.T
bayesian_trans

,hc,mc,lc
P-hC,0.597015,0.391791,0.011194
P-mC,0.050251,0.938023,0.011725
P-lC,0.074074,0.259259,0.666667


In [93]:
first_fold_res = pd.Series(bayesian_trans.values @ sales_count.values, index=bayesian_trans.index)
first_fold_res

P-hC    29328.358209
P-mC    30422.948074
P-lC    31685.185185
dtype: float64

## Second Fold (Research)

In [97]:
sec_fold = preds_total_preds * first_fold_res
sec_fold

P-hC     7860.0
P-mC    18162.5
P-lC     4277.5
dtype: float64

In [99]:
R_hP =  sec_fold.sum()
R_hP

30299.999999999996